# Python для сбора и анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

## Операции с  датафреймами `pandas`:  часть 2

* Добавление новых столбцов
* Группировка и агрегирование

### Напоминание о данных

Импортируем `pandas`:

In [1]:
import pandas as pd

Вспомним, с какими данными мы работаем – мы продолжаем работу с файлом `Salaries.csv` из предыдущей лекции *Операции с  датафреймами pandas:  часть 1*.

В файле `Salaries.csv` сохранены данные по сотрудникам университета в США, а именно следующие их характеристики:

* `rank`: должность;
* `discipline`: тип преподаваемой дисциплины (`A` – теоретическая, `B` – практическая);
* `yrs.since.phd`: число лет с момента получения степени PhD;
* `yrs.service`: число лет опыта работы;
* `sex`: пол;
* `salary`: заработная плата за 9 месяцев, в долларах.

In [2]:
df = pd.read_csv("Salaries.csv")

### Добавление новых столбцов

Так как отдельный столбец датафрейма является объектом типа *pandas Series*, который наследует свойства массива, выполнять операции над столбцами довольно просто. Например, мы хотим добавить в `df` столбец с заработной платой в тысячах. Для этого достаточно выбрать столбец `salary` и поделить все его значения на 1000:

In [3]:
df["salary"] / 1000

0      139.750
1      173.200
2       79.750
3      115.000
4      141.500
        ...   
392    103.106
393    150.564
394    101.738
395     95.329
396     81.035
Name: salary, Length: 397, dtype: float64

Теперь запишем полученный результат в новый столбец `salary_th` датафрейма `df`:

In [4]:
df["salary_th"] = df["salary"] / 1000

Точно так же можно выполнять поэлементные операции с несколькими столбцами. Например, посчитать разность двух столбцов:

In [5]:
df["yrs.service"] - df["yrs.since.phd"] 

0      -1
1      -4
2      -1
3      -6
4       1
       ..
392    -3
393   -12
394   -17
395   -10
396    -4
Length: 397, dtype: int64

По умолчанию новые столбцы записываются в конец датафрейма, но при желании столбцы можно упорядочить по своему желанию.

**Пример:** в некотором датафрейме `df` есть столбцы `a`, `b`, `c`, мы хотим поменять их местами так, чтобы сначала был `c`, потом `a`, а потом `b`:

    cols = ['c', 'a', 'b']
    df = df[cols]

Теперь рассмотрим случай посложнее. Допустим, мы хотим добавить новый столбец `female`, который будет содержать значения 1 (респондент женского пола) и 0 (респондент мужского пола). Сделать это можно разными способами. Первый способ – простой, без написания функций. 

Когда мы проверяем условие для всех значений в столбце, `pandas` возвращает набор из `True` (условие выполняется) и `False` (условие не выполняется):

In [6]:
df["sex"] == "female"

0      False
1      False
2      False
3      False
4      False
       ...  
392    False
393    False
394    False
395    False
396    False
Name: sex, Length: 397, dtype: bool

Для Python вообще и для pandas в частности значение `True` эквивалентно 1, а значение `False` – 0. Воспользуемся этим и вспомним метод `.astype()`, который мы обсуждали в контексте массивов NumPy:

In [7]:
# превратили тип bool в int
# True -> 1, False -> 0

(df["sex"] == "Female").astype(int)

0      0
1      0
2      0
3      0
4      0
      ..
392    0
393    0
394    0
395    0
396    0
Name: sex, Length: 397, dtype: int64

Задача решена, сохраним новый столбец:

In [8]:
df["female"] = (df["sex"] == "Female").astype(int)

df["female"].value_counts()

0    358
1     39
Name: female, dtype: int64

Второй способ решить эту задачу – написать соответствующую функцию и применить её ко всем ячейкам в столбце с помощью метода `.apply()`. В нашем случае функция будет маленькая и явно «разовая», поэтому ее можно не определять через `def`, а создать как анонимную lambda-функцию:

In [9]:
df["female"] = df["sex"].apply(lambda x: int(x == "Female"))
df.head()

,Unnamed: 0,rank,discipline,yrs.since.phd,yrs.service,sex,salary,salary_th,female
0,1,Prof,B,19,18,Male,139750,139.75,0
1,2,Prof,B,20,16,Male,173200,173.20,0
2,3,AsstProf,B,4,3,Male,79750,79.75,0
3,4,Prof,B,45,39,Male,115000,115.00,0
4,5,Prof,B,40,41,Male,141500,141.50,0


Функция `lambda x: int(x == "Female")` принимает на вход какой-то `x`, а возвращает 1 или 0. Метод `.apply()` применяет эту функцию ко всем ячейкам выбранного столбца (аналогично растяжению какой-то ячейки с функцией в Excel или Google Sheets). То есть функцию мы пишем как будто бы для одной ячейки, а затем без всяких циклов применяем ее много раз.

Если функция, которую мы хотим применить, подразумевает большее количество операций, разумнее будет сделать выбор в пользу обычных неанонимных функций, задаваемых через `def`. Напишем функцию `new_rank()`, которая принимает на вход значение должности, а возвращает его числовой эквивалент (1 для `Prof`, 2 для `AssocProf`, 3 для `AsstProf`):

In [10]:
def new_rank(x):
    if x == "Prof":
        y = 1
    elif x == "AssocProf":
        y = 2
    elif x == "AsstProf":
        y = 3
    else:
        y = None
    return y

Почему в функции выше мы добавили ветку с `else`, которая присваивает `y` значение `None`? Для универсальности, на случай, если в столбце встречаются пропущенные значения. В нашем случае это излишне, все ячейки в столбце `rank` заполнены, но в общем случае, если функция столкнется с неучтенным в ней значением, метод `.apply()` не сможет ее применить ко всем ячейкам и вызовет ошибку. А так все неизвестные функции значения будут заменены на пустый ячейки `None` (то же, что `NaN`).

In [11]:
df["rank_num"] = df["rank"].apply(new_rank) 
df.head()

,Unnamed: 0,rank,discipline,yrs.since.phd,yrs.service,sex,salary,salary_th,female,rank_num
0,1,Prof,B,19,18,Male,139750,139.75,0,1
1,2,Prof,B,20,16,Male,173200,173.20,0,1
2,3,AsstProf,B,4,3,Male,79750,79.75,0,3
3,4,Prof,B,45,39,Male,115000,115.00,0,1
4,5,Prof,B,40,41,Male,141500,141.50,0,1


### Группировка и агрегирование

Для начала сгруппируем сотрудников по полу (`sex`). Группировка осуществляется с помощью метода `.groupby()`. Далее, на этот метод можно «наслоить» любую функцию или функции для агрегирования, указав их внутри другого метода `.agg()`. Вычислим средние значения по всем числовым столбцам, отдельно для женщин, отдельно для мужчин:

In [12]:
df.groupby("sex").agg("mean")

,Unnamed: 0,yrs.since.phd,yrs.service,salary,salary_th,female,rank_num
sex,,,,,,,
Female,171.000000,16.512821,11.564103,101002.410256,101.002410,1.0,1.820513
Male,202.050279,22.946927,18.273743,115090.418994,115.090419,0.0,1.463687


**NB.** Более новые версии `pandas` менее лояльны к типам данных, они не будут по умолчанию выбирать только числовые столбцы, для которых вычисление среднего возможно, поэтому код выше не сработает. Нужно явно добавить опцию `numeric_only = True`:

In [13]:
df.groupby("sex").agg("mean", numeric_only = True)

,Unnamed: 0,yrs.since.phd,yrs.service,salary,salary_th,female,rank_num
sex,,,,,,,
Female,171.000000,16.512821,11.564103,101002.410256,101.002410,1.0,1.820513
Male,202.050279,22.946927,18.273743,115090.418994,115.090419,0.0,1.463687


То, что вернул нам метод `.agg()`, является обычным датафреймом, поэтому, если мы хотим получить только среднее по определенному столбцу, мы можем выбрать его по названию:

In [14]:
# средняя заработная плата женщин и мужчин

df.groupby("sex").agg("mean")["salary"]
# df.groupby("sex").agg("mean", numeric_only = True)

sex
Female    101002.410256
Male      115090.418994
Name: salary, dtype: float64

А можем выбрать его и до применения `.agg()`, разумный способ для новых версий pandas, поскольку мы уже заранее выбираем числовой столбец (числовые столбцы) для дальнейших действий.

In [15]:
df.groupby("sex")["salary"].agg("mean")

sex
Female    101002.410256
Male      115090.418994
Name: salary, dtype: float64

Внутри `.agg()` можно указать сразу несколько функций, тогда их надо оформить в виде списка:

In [16]:
# сразу несколько столбцов - список названий
# сразу среднее и медиана

df.groupby("sex")[["salary", "yrs.service"]].agg(["mean", "median"])

salary           yrs.service       
                 mean    median        mean median
sex                                               
Female  101002.410256  103750.0   11.564103   10.0
Male    115090.418994  108043.0   18.273743   18.0

Внутри `.agg()` можно также использовать свою функцию. Напишем, например, функцию `my_range()` для вычисления размаха – меры разброса данных, которая представляет собой разницу между максимальным значением и минимальным:

In [17]:
def my_range(x):
    return max(x) - min(x) 

И применим ее внутри `.agg()` (обратите внимание, тут название уже без кавычек, Python по умолчанию его не знает):

In [18]:
# сразу несколько столбцов - список названий
# сразу min, max и размах

df.groupby("sex")[["salary", "yrs.service"]].agg(["min", "max", my_range])

salary                  yrs.service             
          min     max my_range         min max my_range
sex                                                    
Female  62884  161101    98217           0  36       36
Male    57800  231545   173745           0  60       60

Если мы хотим для одних столбцов вычислить какую-то одну характеристику, а для других – другую, внутри `.agg()` это можно задать с помощью словаря. Так, например, в нашем случае, логичная мера центральной тенденции для заработной платы – это среднее, а для должности в числовом виде – медиана (строго говоря, мода, всего три качественных значения, но раз должности можно упорядочить, медиану можно допустить).

In [19]:
df.groupby("sex").agg({"salary" : "mean", 
                       "rank_num" : "median"})

,salary,rank_num
sex,,
Female,101002.410256,2.0
Male,115090.418994,1.0


В заключение давайте более внимательно посмотрим на объект, который получается в процессе группировки через `.groupby()` и поймем, как его еще можно использовать. 

In [20]:
df.groupby("rank") 

Результат группировки от нас скрыт, он хранится в объекте особого типа `DataFrameGroupBy`. Чтобы посмотреть, что внутри, воспользуемся циклом:

In [21]:
for g in df.groupby("rank"):
    print(g)

('AssocProf',      Unnamed: 0       rank discipline  yrs.since.phd  yrs.service     sex  \
5             6  AssocProf          B              6            6    Male   
10           11  AssocProf          B             12            8    Male   
24           25  AssocProf          A             13            8  Female   
39           40  AssocProf          B              9            9    Male   
41           42  AssocProf          B             23           23    Male   
..          ...        ...        ...            ...          ...     ...   
363         364  AssocProf          A             20           17    Male   
367         368  AssocProf          A             10            1    Male   
370         371  AssocProf          A             13            8    Male   
379         380  AssocProf          A             11            8    Male   
382         383  AssocProf          A              8            5    Male   

     salary  salary_th  female  rank_num  
5     97000     97

Цикл выше выдает нам кортежи, в которых заключены пары значений: название группы и маленький датафрейм со строками, соответствующими этой группе. Для чего это можно использовать? Например, для сохранения данных по каждой группе в отдельный файл. Сделаем перебор в цикле сразу по элементам внутри пары (вспомните словари и перебор по `.items()`):

In [22]:
# на первом месте название, на втором – датафрейм

for group, tab in df.groupby("rank"):
    fname = group + ".xlsx"
    tab.to_excel(fname) 

Код выше перебирает названия групп (`group`) и соответствующие им строки датафрейма (`tab`), забирает названия и доклеивает к ним расширение `.xlsx`. А затем через метод `.to_excel()` выгружает маленькие датафреймы для каждой группы `dat` в файлы с сформированными на предыдущем шаге названиями. Созданные файлы Excel можно найти в рабочей папке, в той папке, где находится текущий ipynb-файл с кодом. 